In [1]:
project_id = 'elife-data-pipeline'
source_dataset = 'de_dev'
output_dataset = 'de_dev'
output_table_prefix = 'data_science_'

max_workers = 10
max_editors = 100
email = 'd.ecer@elifesciences.org'

In [2]:
import logging
from datetime import datetime
from functools import partial
from concurrent.futures import ThreadPoolExecutor
from typing import List

import pandas as pd
from tqdm.auto import tqdm

import data_science_pipeline.configure_warnings  # pylint: disable=unused-import

from data_science_pipeline.utils.europepmc import (
    EuropePMCApi,
    europepmc_requests_retry_session
)
from data_science_pipeline.utils.bq import to_gbq, is_bq_not_found_exception
from data_science_pipeline.utils.jupyter import (
    read_big_query as _read_big_query,
)

In [3]:
LOGGER = logging.getLogger(__name__)

logging.basicConfig(level='INFO')
logging.getLogger('data_science_pipeline').setLevel(logging.INFO)

In [4]:
editor_parsed_pubmed_links_table_name = '{output_dataset}.{prefix}{suffix}'.format(
    output_dataset=output_dataset,
    prefix=output_table_prefix,
    suffix='editor_pubmed_links'
)

editor_parsed_pubmed_ids_table_name = '{output_dataset}.{prefix}{suffix}'.format(
    output_dataset=output_dataset,
    prefix=output_table_prefix,
    suffix='editor_pubmed_ids'
)

In [5]:
read_big_query = partial(_read_big_query, project_id=project_id)

In [6]:
default_query_props = dict(project=project_id, dataset=source_dataset)

In [7]:
_sql = (
    'SELECT person_id FROM `{table}`'
).format(table=editor_parsed_pubmed_ids_table_name)

try:
    existing_editor_ids_df = read_big_query(_sql)
except Exception as e:  # pylint: disable=broad-except
    if not is_bq_not_found_exception(e):
        raise
    print('table not found: %s' % editor_parsed_pubmed_ids_table_name)
    existing_editor_ids_df = pd.DataFrame(columns=['person_id'])
print(len(existing_editor_ids_df))
existing_editor_ids_df.head()

> ```sql
> SELECT person_id FROM `de_dev.data_science_editor_pubmed_ids`
> ```

Downloading: 100%|██████████| 609/609 [00:00<00:00, 2241.40rows/s]

609


,person_id
0,16148
1,31785
2,174689
3,1097
4,1160


In [8]:
existing_editor_ids_set = set(existing_editor_ids_df['person_id'])
len(existing_editor_ids_set)

609

In [9]:
editor_parsed_pubmed_links_df = read_big_query(
    'SELECT * FROM `{table_name}`\nWHERE parsed_search_term IS NOT NULL'.format(
        table_name=editor_parsed_pubmed_links_table_name
    )
)
print(len(editor_parsed_pubmed_links_df))
editor_parsed_pubmed_links_df.head()

> ```sql
> SELECT * FROM `de_dev.data_science_editor_pubmed_links`
> WHERE parsed_search_term IS NOT NULL
> ```

Downloading: 100%|██████████| 611/611 [00:00<00:00, 1154.69rows/s]

611


,name,parsed_search_term,person_id,pubmed_url,resolved_pubmed_url,search_term
0,Yuuki Watanabe,"{'exclude': None, 'include': {'author': ['Wata...",126873,https://www.ncbi.nlm.nih.gov/pubmed/?term=Wata...,https://www.ncbi.nlm.nih.gov/pubmed/?term=Wata...,Watanabe YY[Author]
1,Chima Nwaogu,"{'exclude': None, 'include': {'author': ['Nwao...",178962,https://www.ncbi.nlm.nih.gov/pubmed/?term=Nwao...,https://www.ncbi.nlm.nih.gov/pubmed/?term=Nwao...,Nwaogu CJ[Author]
2,Bernhard Schmid,"{'exclude': None, 'include': {'author': ['Schm...",70207,https://www.ncbi.nlm.nih.gov/pubmed/?term=Schm...,https://www.ncbi.nlm.nih.gov/pubmed/?term=Schm...,Schmid B[Author]
3,Merijn Kant,"{'exclude': None, 'include': {'author': ['Kant...",14193,http://www.ncbi.nlm.nih.gov/pubmed?term=Kant%2...,https://www.ncbi.nlm.nih.gov/pubmed?term=Kant%...,Kant M R[au]
4,Laurent Keller,"{'exclude': None, 'include': {'author': ['Kell...",7970,http://www.ncbi.nlm.nih.gov/pubmed/?term=Kelle...,https://www.ncbi.nlm.nih.gov/pubmed/?term=Kell...,Keller L[Author]


In [10]:
remaining_editor_parsed_pubmed_links_df = editor_parsed_pubmed_links_df[
    ~editor_parsed_pubmed_links_df['person_id'].isin(existing_editor_ids_set)
]
print(len(remaining_editor_parsed_pubmed_links_df))
remaining_editor_parsed_pubmed_links_df.head(3)

0


,name,parsed_search_term,person_id,pubmed_url,resolved_pubmed_url,search_term


In [11]:
processing_editor_parsed_pubmed_links_df = remaining_editor_parsed_pubmed_links_df
if max_editors:
    processing_editor_parsed_pubmed_links_df = processing_editor_parsed_pubmed_links_df[:max_editors]
len(processing_editor_parsed_pubmed_links_df)

0

In [12]:
def get_editor_pubmed_paper_ids(europepmc_api: EuropePMCApi, row) -> List[str]:
    parsed_search_term = row.parsed_search_term
    if not parsed_search_term:
        return None
    author_names = parsed_search_term.get('include', {}).get('author')
    try:
        author_names = parsed_search_term.get('include', {}).get('author')
        return europepmc_api.get_author_pmids(author_names)
    except:  # pylint: disable=bare-except
        LOGGER.error('failed to retrieve pubmed ids for author names: %s', author_names, exc_info=1)
        return None


editor_pubmed_links_result_df = processing_editor_parsed_pubmed_links_df[:max_editors].copy()

with europepmc_requests_retry_session() as session:
    europepmc_api = EuropePMCApi(
        session,
        params={'email': email}
    )
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        editor_pubmed_links_result_df['pubmed_ids'] = list(tqdm(
            executor.map(
                lambda row: get_editor_pubmed_paper_ids(europepmc_api, row),
                editor_pubmed_links_result_df.itertuples()
            ),
            total=len(editor_pubmed_links_result_df),
            leave=False
        ))

In [13]:
non_empty_editor_pubmed_links_result_df = editor_pubmed_links_result_df[
    ~pd.isnull(editor_pubmed_links_result_df['pubmed_ids'])
].copy()
len(non_empty_editor_pubmed_links_result_df)

0

In [14]:
non_empty_editor_pubmed_links_result_df['provenance'] = [{
    'source': 'europepmc',
    'imported_timestamp': datetime.utcnow().isoformat()
}] * len(non_empty_editor_pubmed_links_result_df)

In [15]:
if len(non_empty_editor_pubmed_links_result_df) == 0:
    print('no data to upload')
else:
    print('writing to:', editor_parsed_pubmed_ids_table_name)
    to_gbq(
        non_empty_editor_pubmed_links_result_df,
        project_id=project_id,
        destination_table=editor_parsed_pubmed_ids_table_name,
        if_exists='append'
    )
    print('done')

no data to upload
